# Part 4: XML creation
## Dependencies: OTIO and pandas



In [34]:
#audio
from __future__ import print_function
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import librosa
import librosa.display

In [2]:
import pandas as pd 
import pyloudnorm as pyln

In [3]:
#video
import moviepy
from moviepy.editor import *
import opentimelineio as otio
import argparse

Failed to establish dbus connection

In [4]:
import cv2

### Quick summary, OTIO can output XML that CAN be used to import timelines. We just need to learn the behavior and then create a loop to automate clipping, using a imported dataframe as a outline.

# Timeline Outline behavior testing
- First create a list of videos
    - sfs
- Sec create list of audio

- Link audio to video, IE if mic1 use center camera

## Use the lists of camera and audio to createa loops creating timeranges

In [5]:
VIDEO_FILEPATH_LIST= ["Batman Middle.mp4","Batman Right.mp4","Batman Left.mp4"]

# "Batman Right.mp4" is 1   ||  "Batman Left.mp4" is 2
AUDIO_FILEPATH_LIST=["Data/Audio/Osi Audio Extracted.wav", "Data/Audio/Scott Audio Extracted.wav", \
               "Data/Audio/Chukwu Audio Extracted.wav", "Data/Audio/Crystal Audio Extracted.wav"]

In [6]:
AUDIO_VIDEO_TUPLE_LIST=[("Osi Audio Extracted.wav", 1), ("Scott Audio Extracted.wav",1), \
               ("Chukwu Audio Extracted.wav",2), ("Crystal Audio Extracted.wav",2)]

In [7]:

# build the structure
tl = otio.schema.Timeline(name="Example timeline")

# add track for each video file and each audio file
#for each file add a track
for i in VIDEO_FILEPATH_LIST :
    tr = otio.schema.Track(name=i)
    tl.tracks.append(tr)

for i in AUDIO_VIDEO_TUPLE_LIST:
    tr = otio.schema.Track(name=i[0])
    tl.tracks.append(tr)

In [8]:
#testing if the tracks are added
#otio.adapters.write_to_file(tl, 'Baretimeline.otio')

# With the timeline and tracks set, now is the time to match add clips to each of them, Fairly skeleton based

In [9]:
#import data with the following: 
#audio idxmax (we will use this to link the audio video tuple list), 
#the  list of tuples/intervals 


# Further data editing 

## Create a camera view column that matches the audio_video_tuple to  idxmax

In [10]:
data_df=pd.read_pickle('idxmax.pkl')

In [11]:
#create new column linking idxmax to Video File list view
data_df['cam_view']= data_df['idxmax'].apply(lambda row: AUDIO_VIDEO_TUPLE_LIST[row-1][1] if row!=0 else 0)

In [12]:
#use indexes of when crossover is 1 to change cam_view to 0 for three seconds after crossover

In [13]:

cap = cv2. VideoCapture("Batman Left.mp4")
vlength = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
alength=len(data_df)

## Create intervals of data in tuple form. 
## Audio Tuples represent the start and end frame of when mic is dominant. Camera 0 is default cam in case of uncertainty, which is center cam in this scenario.
## Video Tuples represent the start and end frame of which mic to use. 


In [14]:
def dataframe_getintervals(series,desiredvalue):
    #make sure series is the df['column']
    t=series.index[series==desiredvalue].to_series()
    interval_list=t.groupby(t.diff().ne(1).cumsum()).agg(['first','last']).apply(tuple,1).tolist()
    
    return interval_list

In [15]:
#create lists of tuples for each camera based on the audio

list_of_idxmax_mic_data=[]
for i in range(len(AUDIO_VIDEO_TUPLE_LIST)+1):
    tuple_list=dataframe_getintervals(data_df['idxmax'],i)
    list_of_idxmax_mic_data.append(tuple_list)

    
##create lists of tuples for each Audio

# Variables and their definitions

```py 
list_of_idxmax_mic_data  
```
- List of lists of tuples defining the (starting frame, ending frame)
- Position on the list describes which audio file is used, where 0 is unsure
- IE: list_of_idxmax_mic_data[1] is the (starting frame, ending frame) of Audio file 1



```py 
VIDEO_FILEPATH_LIST  
```
- List of video filepaths. 
- We will use the list indexes to reference audio video pairs in the next variable

```py 
AUDIO_VIDEO_TUPLE_LIST 
```
- A Tuple of (Audio Filepath, VIDEO_FILEPATH_LIST index of associated audio)
- use this to connect a audio to video track

In [28]:
#skeletal outline of otio


# build the structure
tl = otio.schema.Timeline(name="Example timeline")

# add track for each video file and each audio file
#for each file add a track

#create lists for each track to reference back to later
#vtr is video track, etc.
vtr_list=[]
atr_list=[]


#add a audio AND video track for each video track
    #default cam first because it is lowest priority
vtr_default = otio.schema.Track(name="Default_camera", kind="Video")
tl.tracks.append(vtr_default)

for i in AUDIO_VIDEO_TUPLE_LIST:
    atr = otio.schema.Track(name=i[0], kind="Audio")
    tl.tracks.append(atr)
    atr_list.append(atr)
    
    #video
    vtr = otio.schema.Track(name=i[0]+"_video", kind="Video")
    tl.tracks.append(vtr)
    vtr_list.append(vtr)



        v_clip_available_range=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(0, vrate),
                duration=otio.opentime.RationalTime(vlength, vrate))

### Available range returns the specified TimeRange available via the Clip’s media_reference. This should tell you how long that movie is and what timecode it starts at. For example: “wedding.mov” starts at timecode 01:00:00:00 and is 30 minutes long.
        vclip_ref = otio.schema.ExternalReference(target_url=vfname,
            available_range=vclip_timerange)
        
        
        vcl = otio.schema.Clip(
                    name="vClip{}".format(i2 + 1),
                    media_reference=vclip_ref,

###  The Source range  of a Clip will trim the Clip to only that range of media.
                    # available_range_from_list is the 
                    source_range=otio.opentime.TimeRange(
                        start_time=otio.opentime.RationalTime(
                            vclip_timerange.start_time.value,
                            vclip_timerange.start_time.rate
                        ),
                        duration=otio.opentime.RationalTime(
                            vclip_timerange.duration.value,
                            vclip_timerange.duration.rate
                        ),
                    )
                )
        
        vtrack.append(vcl)
        
        
        vstartingframe=0
        astartingframe=0
        

In [29]:
vtr_list[1]

otio.schema.Track(name='Scott Audio Extracted.wav_video', children=[], source_range=None, metadata={})

In [30]:
vrate=24
arate=500
arate_actual=24

# Two different loops, One for Video, another for audio

In [31]:
# i is to keep track which audio file we are currently on
i=-1

# i2 is to keep track of how many clips there are
i2=0

for lists_of_tuples in list_of_idxmax_mic_data:
    #connect the list of tuples to the audio file
    #ignore 0 for now. We will come back to this later, i will still be -1 but we will add a default clip instead
    
    if i==-1:
    #add the entire default cam clip in the lowest priority
        #vclip
        #aclip
        pass
    
    else: 
        #print(AUDIO_VIDEO_TUPLE_LIST[i],VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]])
    
        # Connect the audio and video tracks 
        atrack=atr_list[i]
        afname=AUDIO_VIDEO_TUPLE_LIST[i][0]
        
        # AUDIO_VIDEO_TUPLE_LIST[i][1] references the VIDEO_FILEPATH_LIST to determine which video file to use
        vtrack=vtr_list[i]
        vfname=VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]]
        
        
        
        #will remember the previous end of clip's timecode
        #will reset to 0 when a video-audio track pair is done
        vprevious_end_timecode=0
        aprevious_end_timecode=0
        
    #=========================================================================
           
        for tuples in lists_of_tuples:
            
        #=====================================================
          #adding variables for time and duration calculation
            #Video clips 
            v_clip_starttime=tuples[0]/arate*vrate
            v_clip_duration=tuples[1]/arate*vrate-v_clip_starttime

            
        #Specifying the start time and end time of the video file where clips come from
            v_clip_available_range=otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, vrate),
            duration=otio.opentime.RationalTime(vlength, vrate))
            
            vref = otio.schema.ExternalReference(target_url=vfname,
            available_range=v_clip_available_range)
            
            
        #specifying where the start timecode and end timecode of the clip is
            v_clip_source_range=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(v_clip_starttime, vrate),
                duration=otio.opentime.RationalTime(v_clip_duration, vrate))



        #Video Gaps
            #the gap length is equal to (timecode of next clip time - timecode end of previous clip)
            v_gap_start_time=0
            v_gap_duration=v_clip_starttime-vprevious_end_timecode
            
            v_gap_timerange=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(v_gap_start_time, vrate),
                duration=otio.opentime.RationalTime(v_gap_duration, vrate))
            
                
            
            
         #=====================================================   
            # adding gaps before adding clips
            #convert the tuple ranges from audio rate to video rate
                #audio rate=500
                #video rate=24



            # create gap settings
            vgap = otio.schema.Gap(
                name="vGap{}".format(i2 + 1),

                # available_range_from_list is the 
                source_range=otio.opentime.TimeRange(
                    start_time=otio.opentime.RationalTime(
                        v_gap_timerange.start_time.value,
                        v_gap_timerange.start_time.rate
                    ),
                    duration=otio.opentime.RationalTime(
                        v_gap_timerange.duration.value,
                        v_gap_timerange.duration.rate
                    ),
                )
            )

            # put the clip into the track
            vtrack.append(vgap)



        #=======================================================
        # adding Video clips 
        #convert the tuple ranges from audio rate to video rate
        #audio rate=500

                        #add clip to track                

            vcl = otio.schema.Clip(
                        name="vClip{}".format(i2 + 1),
                        media_reference=vref,

                        # available_range_from_list is the 
                        source_range=otio.opentime.TimeRange(
                            start_time=otio.opentime.RationalTime(
                                v_clip_source_range.start_time.value,
                                v_clip_source_range.start_time.rate
                            ),
                            duration=otio.opentime.RationalTime(
                                v_clip_source_range.duration.value,
                                v_clip_source_range.duration.rate
                            ),
                        )
                    )

            vtrack.append(vcl)

            vprevious_end_timecode=tuples[1]/arate*vrate
            #apreviousduration=
        


            i2=i2+1

            
    i=i+1
    #if i == 2:
        #break

In [32]:
# i is to keep track which audio file we are currently on
i=-1

# i2 is to keep track of how many clips there are
i2=0

for lists_of_tuples in list_of_idxmax_mic_data:
    #connect the list of tuples to the audio file
    #ignore 0 for now. We will come back to this later, i will still be -1 but we will add a default clip instead
    
    if i==-1:
    #add the entire default cam clip in the lowest priority
        #vclip
        #aclip
        pass
    
    else: 
        print(AUDIO_VIDEO_TUPLE_LIST[i],VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]])
    
        # Connect the audio and video tracks 
        atrack=atr_list[i]
        afname=AUDIO_VIDEO_TUPLE_LIST[i][0]
        
        # AUDIO_VIDEO_TUPLE_LIST[i][1] references the VIDEO_FILEPATH_LIST to determine which video file to use
        vtrack=vtr_list[i]
        vfname=VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]]
        
        
        
        #will remember the previous end of clip's timecode
        #will reset to 0 when a video-audio track pair is done
        vprevious_end_timecode=0
        aprevious_end_timecode=0
        
    #=========================================================================
           
        for tuples in lists_of_tuples:
            
        #=====================================================
          #adding variables for time and duration calculation
            #Video clips 
            a_clip_starttime=tuples[0]/arate*arate_actual
            a_clip_duration=tuples[1]/arate*arate_actual-a_clip_starttime

            
        #Specifying the start time and end time of the video file where clips come from
            a_clip_available_range=otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, arate_actual),
            duration=otio.opentime.RationalTime(alength*arate_actual, arate_actual))
            
            aref = otio.schema.ExternalReference(target_url=afname,
            available_range=a_clip_available_range)
            
            
        #specifying where the start timecode and end timecode of the clip is
            a_clip_source_range=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(a_clip_starttime, arate_actual),
                duration=otio.opentime.RationalTime(a_clip_duration, arate_actual))



        #Video Gaps
            #the gap length is equal to (timecode of next clip time - timecode end of previous clip)
            a_gap_start_time=0
            a_gap_duration=a_clip_starttime-aprevious_end_timecode
            
            a_gap_timerange=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(a_gap_start_time, arate_actual),
                duration=otio.opentime.RationalTime(a_gap_duration, arate_actual))
            
                
            
            
         #=====================================================   
            # adding gaps before adding clips
            #convert the tuple ranges from audio rate to video rate
                #audio rate=500
                #video rate=24



            # create gap settings
            agap = otio.schema.Gap(
                name="vGap{}".format(i2 + 1),

                # available_range_from_list is the 
                source_range=otio.opentime.TimeRange(
                    start_time=otio.opentime.RationalTime(
                        a_gap_timerange.start_time.value,
                        a_gap_timerange.start_time.rate
                    ),
                    duration=otio.opentime.RationalTime(
                        a_gap_timerange.duration.value,
                        a_gap_timerange.duration.rate
                    ),
                )
            )

            # put the clip into the track
            atrack.append(agap)



        #=======================================================
        # adding Video clips 
        #convert the tuple ranges from audio rate to video rate
        #audio rate=500

                        #add clip to track                

            acl = otio.schema.Clip(
                        name="aClip{}".format(i2 + 1),
                        media_reference=aref,

                        # available_range_from_list is the 
                        source_range=otio.opentime.TimeRange(
                            start_time=otio.opentime.RationalTime(
                                a_clip_source_range.start_time.value,
                                a_clip_source_range.start_time.rate
                            ),
                            duration=otio.opentime.RationalTime(
                                a_clip_source_range.duration.value,
                                a_clip_source_range.duration.rate
                            ),
                        )
                    )

            atrack.append(acl)

            aprevious_end_timecode=tuples[1]/arate*arate_actual
            #apreviousduration=
        


            i2=i2+1

            
    i=i+1
    #if i == 2:
        #break

('Osi Audio Extracted.wav', 1) Batman Right.mp4
('Scott Audio Extracted.wav', 1) Batman Right.mp4
('Chukwu Audio Extracted.wav', 2) Batman Left.mp4
('Crystal Audio Extracted.wav', 2) Batman Left.mp4


In [35]:
otio.adapters.write_to_file(tl, 'videoaudio_beta1.otio')

'videoaudio_beta1.xml'

# List of starting time of being the dominant speaker and the duration
```py
('Data/Audio/Osi Audio Extracted.wav', 1)
('Data/Audio/Osi Audio Extracted.wav', 1) Batman Right.mp4
5.0 5.814
52.256 68.582
150.312 24.148
330.572 33.254
495.476 11.958
508.292 4.256
631.932 2.57
672.094 16.796
716.55 6.716
739.93 8.774
753.63 6.882
781.768 4.996
848.5 2.53
855.804 14.33
915.096 95.554
1054.966 18.062
1131.93 3.532
1250.422 1.634
1265.846 8.658
1278.098 19.812
('Data/Audio/Scott Audio Extracted.wav', 1) Batman Right.mp4
29.878 13.236
44.242 5.452
120.84 29.47
253.812 2.886
507.436 0.854
538.042 2.076
618.74 4.106
634.504 6.394
688.892 16.592
711.148 5.4
723.268 2.732
748.706 4.922
895.318 16.53
1010.652 9.78
1025.558 12.08
1215.542 34.878
1258.498 7.346
1297.912 1.248
('Data/Audio/Chukwu Audio Extracted.wav', 2) Batman Left.mp4
10.816 12.376
43.116 1.124
49.696 2.558
247.8 6.01
256.7 26.724
287.506 6.202
370.096 45.72
453.658 4.122
512.55 25.49
540.12 41.926
589.822 19.068
629.012 2.918
706.598 4.548
726.002 6.506
760.514 13.74
778.346 3.42
786.766 27.816
1020.924 4.632
1073.03 34.318
1185.776 15.498
1208.034 7.506
1299.354 3.456
('Data/Audio/Crystal Audio Extracted.wav', 2) Batman Left.mp4
23.194 6.682
174.462 73.336
283.426 4.078
293.71 36.86
363.828 6.266
415.818 37.838
457.782 37.692
582.048 7.772
608.892 9.846
622.848 6.162
640.9 31.192
705.486 1.11
732.51 7.418
774.256 4.088
814.584 33.914
851.032 4.77
870.136 25.18
911.85 3.244
1020.434 0.488
1037.64 17.324
1107.35 24.578
1135.464 50.31
1201.276 6.756
1252.058 6.438
1274.506 3.59
1299.162 0.19
```

In [28]:
import cv2
cap = cv2. VideoCapture("Batman Left.mp4")
vlength = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
alength=len(data_df)

# The clip adding via tuple looping

In [52]:
# i is to keep track which audio file we are currently on
i=-1

# i2 is to keep track of how many clips there are
i2=0

for lists_of_tuples in list_of_idxmax_mic_data:
    #connect the list of tuples to the audio file
    #ignore 0 for now. We will come back to this later, i will still be -1 but we will add a default clip instead
    
    if i==-1:
    #add the entire default cam clip in the lowest priority
        #vclip
        #aclip
        pass
    
    else: 
        print(AUDIO_VIDEO_TUPLE_LIST[i],VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]])
    
        # Connect the audio and video tracks 
        atrack=atr_list[i]
        afname=AUDIO_VIDEO_TUPLE_LIST[i][0]
        
        # AUDIO_VIDEO_TUPLE_LIST[i][1] references the VIDEO_FILEPATH_LIST to determine which video file to use
        vtrack=vtr_list[i]
        vfname=VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]]
        
        
        
        #will remember the previous end of clip's timecode
        #will reset to 0 when a video-audio track pair is done
        vprevious_end_timecode=0
        aprevious_end_timecode=0
        
    #=========================================================================
           
        for tuples in lists_of_tuples:
            
        #=====================================================
          #adding variables for time and duration calculation
            #Video clips 
            v_clip_starttime=tuples[0]/arate*vrate
            v_clip_duration=tuples[1]/arate*vrate-v_clip_starttime

            
        #Specifying the start time and end time of the video file where clips come from
            v_clip_available_range=otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, vrate),
            duration=otio.opentime.RationalTime(vlength, vrate))
            
            vref = otio.schema.ExternalReference(target_url=vfname,
            available_range=v_clip_available_range)
            
            
        #specifying where the start timecode and end timecode of the clip is
            v_clip_source_range=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(v_clip_starttime, vrate),
                duration=otio.opentime.RationalTime(v_clip_duration, vrate))



        #Video Gaps
            #the gap length is equal to (timecode of next clip time - timecode end of previous clip)
            v_gap_start_time=0
            v_gap_duration=v_clip_starttime-vprevious_end_timecode
            
            v_gap_timerange=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(v_gap_start_time, vrate),
                duration=otio.opentime.RationalTime(v_gap_duration, vrate))
            
                
            
            
         #=====================================================   
            # adding gaps before adding clips
            #convert the tuple ranges from audio rate to video rate
                #audio rate=500
                #video rate=24



            # create gap settings
            vgap = otio.schema.Gap(
                name="vGap{}".format(i2 + 1),

                # available_range_from_list is the 
                source_range=otio.opentime.TimeRange(
                    start_time=otio.opentime.RationalTime(
                        v_gap_timerange.start_time.value,
                        v_gap_timerange.start_time.rate
                    ),
                    duration=otio.opentime.RationalTime(
                        v_gap_timerange.duration.value,
                        v_gap_timerange.duration.rate
                    ),
                )
            )

            # put the clip into the track
            vtrack.append(vgap)



        #=======================================================
        # adding Video clips 
        #convert the tuple ranges from audio rate to video rate
        #audio rate=500

                        #add clip to track                

            vcl = otio.schema.Clip(
                        name="vClip{}".format(i2 + 1),
                        media_reference=vref,

                        # available_range_from_list is the 
                        source_range=otio.opentime.TimeRange(
                            start_time=otio.opentime.RationalTime(
                                v_clip_source_range.start_time.value,
                                v_clip_source_range.start_time.rate
                            ),
                            duration=otio.opentime.RationalTime(
                                v_clip_source_range.duration.value,
                                v_clip_source_range.duration.rate
                            ),
                        )
                    )

            vtrack.append(vcl)

            vprevious_end_timecode=tuples[1]/arate*vrate
            #apreviousduration=
        


            i2=i2+1

            
    i=i+1
    #if i == 2:
        #break

('Data/Audio/Osi Audio Extracted.wav', 1) Batman Right.mp4
('Data/Audio/Scott Audio Extracted.wav', 1) Batman Right.mp4
('Data/Audio/Chukwu Audio Extracted.wav', 2) Batman Left.mp4
('Data/Audio/Crystal Audio Extracted.wav', 2) Batman Left.mp4


In [53]:
v_gap_duration

505.5839999999989

In [54]:
otio.adapters.write_to_file(tl, 'videoclipsonly.otio')

True

In [ ]:
#experiment analysis Duration is NOT DURATION. it is the end time!
# start time 2400 is a hundered seconds in
# duration 4800 is that the clip goes for 2400 until it reaches 4800 at the end time 

In [ ]:
otio.schema.Clip()

In [124]:
vtr_list[0]

otio.schema.Track(name='Data/Audio/Osi Audio Extracted.wav_video', children=[otio.schema.Gap(name='vGap1', source_range=otio.opentime.TimeRange(start_time=otio.opentime.RationalTime(value=0, rate=24), duration=otio.opentime.RationalTime(value=120, rate=24)), effects=[], markers=[], metadata={}), otio.schema.Clip(name='vClip1', media_reference=otio.schema.ExternalReference(target_url='Batman Right.mp4'), source_range=otio.opentime.TimeRange(start_time=otio.opentime.RationalTime(value=120, rate=24), duration=otio.opentime.RationalTime(value=259.536, rate=24)), metadata={}), otio.schema.Gap(name='vGap2', source_range=otio.opentime.TimeRange(start_time=otio.opentime.RationalTime(value=0, rate=24), duration=otio.opentime.RationalTime(value=994.608, rate=24)), effects=[], markers=[], metadata={}), otio.schema.Clip(name='vClip2', media_reference=otio.schema.ExternalReference(target_url='Batman Right.mp4'), source_range=otio.opentime.TimeRange(start_time=otio.opentime.RationalTime(value=1254.1

In [ ]:
xml_to_otio_test=otio.adapters.read_from_file('sc')
otio.adapters.write_to_file(xml_to_otio_test, "tests.otio")

In [26]:
otio.schema.

otio.schema.Timeline(name='', tracks=otio.schema.Stack(name='tracks', children=[], source_range=None, metadata={}))